In [1]:
!pip install -r requirements.txt


   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.3 MB ? eta -:--:--
   ------------------- -------------------- 2.1/4.3 MB 10.7 MB/s eta 0:00:01
   ----------------------------- ---------- 3.1/4.3 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------- 4.3/4.3 MB 6.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'

In [3]:
import PyPDF2
import glob
import docx
import nltk
import transformers
import torch

c:\Users\memo_\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import os

# Transformers'ın TensorFlow modüllerini yüklemesini tamamen engelliyoruz:
os.environ["USE_TF"] = "0"
os.environ["TRANSFORMERS_NO_TF"] = "1"

print("TensorFlow'u devre dışı bırakma ayarları yapıldı.")
print(f"USE_TF: {os.getenv('USE_TF')}")
print(f"TRANSFORMERS_NO_TF: {os.getenv('TRANSFORMERS_NO_TF')}")

try:
    from nltk.tokenize import word_tokenize
    from sentence_transformers import SentenceTransformer
    from pypdf import PdfReader
    from qdrant_client import QdrantClient
    from qdrant_client.http.models import Distance, VectorParams
    from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
    print("Tüm kütüphaneler başarıyla import edildi!")
except Exception as e:
    print(f"Import sırasında bir hata oluştu: {e}")
    import traceback
    traceback.print_exc()

TensorFlow'u devre dışı bırakma ayarları yapıldı.
USE_TF: 0
TRANSFORMERS_NO_TF: 1
Tüm kütüphaneler başarıyla import edildi!


In [ ]:
# --- 1. Doküman Parsers ---
def parse_pdf(path):
    reader = PdfReader(path)
    return "\n".join(page.extract_text() or "" for page in reader.pages)

def parse_docx(path):
    doc = docx.Document(path)
    return "\n".join(p.text for p in doc.paragraphs)

def parse_txt(path):
    with open(path, encoding="utf-8") as f:
        return f.read()

# Klasördeki tüm dokümanları oku
docs = {}
for path in glob.glob("dosyayolu", recursive=True): # dosyayolu yerine gerçek klasör yolunu yazın
    if path.endswith(".pdf"): docs[path] = parse_pdf(path)
    elif path.endswith(".docx"): docs[path] = parse_docx(path)
    elif path.endswith(".txt"): docs[path] = parse_txt(path)


In [ ]:
# --- 2. Metin Parçalama ---
nltk.download("punkt")
chunks = [] # Orijinal chunk metinlerini burada saklayacağız
metadata_for_qdrant = [] # Qdrant'a yüklenecek metadata

global_chunk_id = 0 # Qdrant için benzersiz ve sıralı ID

for path, text in docs.items():
    if not text or not text.strip():
        continue

    words = word_tokenize(text)
    max_tokens = 300
    chunk_id_in_file_counter = 0 # Her dosya için chunk ID'si

    for i in range(0, len(words), max_tokens):
        # Bu, embedding için kullanılacak chunk metni
        current_chunk_text = " ".join(words[i : i + max_tokens])

        if not current_chunk_text.strip():
            continue

        chunks.append(current_chunk_text) # Bu liste hem embedding hem de retrieve için kullanılacak
        metadata_for_qdrant.append({
            "source": path,
            "chunk_id_in_file": chunk_id_in_file_counter, # Dosya içindeki chunk sırası
            "original_text": current_chunk_text
        })
        global_chunk_id += 1
        chunk_id_in_file_counter += 1

if not chunks:
    print("Hiç chunk üretilemedi. İşlem durduruluyor.")
    exit()

print(f"Toplam {len(chunks)} adet chunk metni oluşturuldu.")

Toplam 3 adet chunk metni oluşturuldu.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\memo_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
#!pip uninstall -y tensorflow tensorflow-cpu tensorflow-gpu tensorboard tb-nightly tensorflow-estimator tensorflow-io tensorflow-io-gcs-filesystem tf-keras keras

In [8]:
from sentence_transformers import SentenceTransformer

#embed_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
embed_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

# Tüm chunk’lar için embedding üret
embeddings = embed_model.encode(chunks, show_progress_bar=True, convert_to_numpy=True)


Batches: 100%|██████████| 1/1 [00:02<00:00,  2.72s/it]


In [ ]:
# 4. Qdrant Bağlantısı ve İndeksleme 
qdrant_client = QdrantClient(host="localhost", port=6333)

collection_name = "llmfao_collection"

vector_size = embeddings.shape[1]

try:
    qdrant_client.delete_collection(collection_name=collection_name)
    print(f"'{collection_name}' koleksiyonu silindi.")
except Exception:
    print(f"'{collection_name}' koleksiyonu silinemedi veya zaten yoktu.")

# Sonra koleksiyonu oluştur
qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
)
print(f"'{collection_name}' koleksiyonu oluşturuldu/yeniden oluşturuldu.")

# Upsert
points = [
    {"id": idx, 
     "vector": embeddings[idx].tolist(), 
     "payload": metadata_for_qdrant[idx]}
    for idx in range(len(embeddings))
]
qdrant_client.upsert(collection_name=collection_name, points=points)
print(f"{len(points)} nokta başarıyla Qdrant'taki '{collection_name}' koleksiyonuna yüklendi.") 

'llmfao_collection' koleksiyonu silindi.
'llmfao_collection' koleksiyonu oluşturuldu/yeniden oluşturuldu.
3 nokta başarıyla Qdrant'taki 'llmfao_collection' koleksiyonuna yüklendi.


In [ ]:
# 5. Retrieve Fonksiyonu (en yakın 5 sonuç) 
def retrieve_top_k(query: str, k: int = 5):
    q_emb = embed_model.encode([query], convert_to_numpy=True)[0]
    hits = qdrant_client.search(
        collection_name=collection_name,
        query_vector=q_emb.tolist(),
        limit=k,
    )
    retrieved_results = []
    for hit in hits:
        source = hit.payload.get("source", "Kaynak bilinmiyor")
        chunk_id_val = hit.payload.get("chunk_id", -1) 
        text_content = hit.payload.get("original_text", "Metin bulunamadı")

        retrieved_results.append({
            "source": source,
            "chunk_id": chunk_id_val,
            "score": hit.score,
            "text": text_content,
            "qdrant_id": hit.id
        })
    return retrieved_results

In [ ]:
#gpt-2

"""model_id = "ytu-ce-cosmos/turkish-gpt2-large"

# 1) Tokenizer yükleme
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 2) Model yükleme
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# 3) Text-generation pipeline oluşturma
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer"""
)

Some parameters are on the meta device because they were offloaded to the cpu and disk.
Device set to use cpu


In [ ]:
#gemma-9b
model_id = "ytu-ce-cosmos/Turkish-Gemma-9b-v0.1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

print(f"'{model_id}' için tokenizer yükleniyor...")

try:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    print("Tokenizer başarıyla yüklendi.")

    print(f"'{model_id}' modeli BitsAndBytesConfig ile yükleniyor...")
    # torch_dtype'ı from_pretrained içinde belirtmek daha doğru olabilir
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=quantization_config,
        torch_dtype=torch.bfloat16, # Modelin ana veri tipi
        device_map="auto", # Accelerate'in modeli dağıtmasına izin ver
        # max_memory parametresini burada da deneyebilirsiniz:
        # max_memory={0: "8GiB", "cpu": "24GiB"} # GPU ve CPU limitlerini ayarlayın
    )
    print("Model başarıyla yüklendi.")

    print("Pipeline oluşturuluyor...")
    # Değişiklik: 'pipeline' değişkeninin adını 'text_generator' olarak değiştirdik
    text_generator = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device=0
    )
    print("Pipeline başarıyla oluşturuldu.")
except Exception as e:
    print(f"Model yükleme veya pipeline oluşturma sırasında bir hata oluştu: {e}")

'model_id = "ytu-ce-cosmos/Turkish-Gemma-9b-v0.1"\n\nquantization_config = BitsAndBytesConfig(\n    load_in_4bit=True,\n    bnb_4bit_quant_type="nf4",\n    bnb_4bit_use_double_quant=True,\n    bnb_4bit_compute_dtype=torch.bfloat16\n)\n\nprint(f"\'{model_id}\' için tokenizer yükleniyor...")\n\ntry:\n    tokenizer = AutoTokenizer.from_pretrained(model_id)\n    print("Tokenizer başarıyla yüklendi.")\n\n    print(f"\'{model_id}\' modeli BitsAndBytesConfig ile yükleniyor...")\n    # torch_dtype\'ı from_pretrained içinde belirtmek daha doğru olabilir\n    model = AutoModelForCausalLM.from_pretrained(\n        model_id,\n        quantization_config=quantization_config,\n        torch_dtype=torch.bfloat16, # Modelin ana veri tipi\n        device_map="auto", # Accelerate\'in modeli dağıtmasına izin ver\n        # max_memory parametresini burada da deneyebilirsiniz:\n        # max_memory={0: "8GiB", "cpu": "24GiB"} # GPU ve CPU limitlerini ayarlayın\n    )\n    print("Model başarıyla yüklendi.")

In [ ]:
#gemma 9b için

def rag_answer(query: str):
    # Retrieve
    results = retrieve_top_k(query, k=5)
    # Kaynak metinleri oluştur
    sources_text = "\n\n".join(
        f"[{i+1}] Skor: {r['score']:.4f}\n{r['text']}"
        for i, r in enumerate(results)
    )
    # Prompt
    system_prompt = "Sen Türkçe NLP uzmanısın. Aşağıdaki kaynaklara dayanarak soruyu yanıtla."
    user_prompt = f"Soru: {query}\n\nKaynaklar:\n{sources_text}\n\nCevabı açıkla:"

    full_prompt = system_prompt + "\n" + user_prompt

    # Cevap üret
    outputs = text_generator(
        full_prompt,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    return outputs[0]["generated_text"]

In [ ]:
#gpt-2 için

#def rag_answer(query: str) -> str:
"""   # 1) Retrieve
    results = retrieve_top_k(query, k=5)

    # 2) Kaynak metinlerini ve skorları birleştir
    sources = [
        f"[{i+1}] Skor: {r['score']:.4f}\n{r['text']}"
        for i, r in enumerate(results)
    ]

    # 3) Model ve tokenizer bilgi
    max_pos = model.config.max_position_embeddings   # örn. 1024
    max_new = 512                                    # üretilecek token
    reserved = 50                                    # sistem+format için küçük pay

    # 4) Soru + format token ihtiyacı
    prefix = (
        "Sen Türkçe NLP uzmanısın. Aşağıdaki kaynaklara dayanarak soruyu yanıtla:\n\n"
        f"Soru: {query}\n\n"
        "Kaynaklar:\n"
    )
    prefix_ids = tokenizer(prefix, return_tensors="pt").input_ids[0]
    prefix_len = len(prefix_ids)

    # 5) Kaynak metinlerini bütçeye göre ekle
    budget = max_pos - max_new - prefix_len - reserved
    included = []
    total = 0
    for s in sources:
        tok = tokenizer(s, return_tensors="pt").input_ids[0]
        if total + len(tok) <= budget:
            included.append(s)
            total += len(tok)
        else:
            break

    sources_text = "\n\n".join(included)

    # 6) Tam prompt
    prompt = prefix + sources_text + "\n\nCevabı açıkla:"

    # 7) Üretim
    output = text_generator(
        prompt,
        max_new_tokens=max_new,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )
    return output[0]["generated_text"]"""


In [ ]:
# --- 7. Örnek Kullanım ---
if __name__ == "__main__":
    question = "Metinde referans alınan kaynaklar nelerdir?"
    answer = rag_answer(question)
    print(answer)
